In [1]:
import pandas as pd
import os
import random
from tqdm import tqdm
import numpy as np
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(color_codes=True)
import pickle

In [3]:
data_dir = '/opt/ml/project/data/' # 경로는 상황에 맞춰서 수정해주세요!
csv_file_path = os.path.join(data_dir, 'total_data_v2.csv') # 데이터는 대회홈페이지에서 받아주세요 :)
df = pd.read_csv(csv_file_path, parse_dates=['Timestamp']) 
df = df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [4]:
np.zeros(len(df))

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
def calculate_accuracy_trend(df):
    for window_size in [3, 5, 10, 30, 50, 100, 200]:
        user_df = df.groupby(df['userID']).shift(1)
        accuracy_trend = user_df.groupby(df['userID']).answerCode.rolling(window=window_size, min_periods=1).mean()
        correct_trend = user_df.groupby(df['userID']).answerCode.rolling(window=window_size, min_periods=1).sum()
        
        df[f'accuracy_trend{window_size}'] = accuracy_trend.values
        df[f'normalized_accuracy_trend{window_size}'] = df[f'accuracy_trend{window_size}'] - df['question_mean']
        df[f'correct_trend{window_size}'] = correct_trend.values
    
    df['shift'] = df.groupby('userID').answerCode.shift(1)
    
    for window_size in [3, 5, 10, 30, 50, 100, 200]:
        user_df = df.groupby(df['userID']).shift(1)
        accuracy_trend = user_df.groupby(df['userID']).answerCode.rolling(window=window_size, min_periods=1).mean()
        correct_trend = user_df.groupby(df['userID']).answerCode.rolling(window=window_size, min_periods=1).sum()
        
        df[f'accuracy_trend{window_size}'] = accuracy_trend.values
        df[f'normalized_accuracy_trend{window_size}'] = df[f'accuracy_trend{window_size}'] - df['question_mean']
        df[f'correct_trend{window_size}'] = correct_trend.values
    
    for window_size in ['1h', '5h', '10h', '1D', '5D', '10D']:
        temp_accuracy_arr = np.zeros(len(df))
        temp_correct_arr = np.zeros(len(df))

        for user_id, temp_df in df.groupby('userID'):
            idx = temp_df.index
            accuracy_time_trend = temp_df.set_index('Timestamp')['shift'].rolling(window_size, min_periods=1).mean()
            correct_time_trend = temp_df.set_index('Timestamp')['shift'].rolling(window_size, min_periods=1).sum()
            temp_accuracy_arr[idx] = accuracy_time_trend
            temp_correct_arr[idx] = correct_time_trend

        df[f'accuracy_time_trend{window_size}'] = temp_accuracy_arr
        df[f'normalized_accuracy_time_trend{window_size}'] = temp_accuracy_arr - df['question_mean'].values
        df[f'correct_time_trend{window_size}'] = temp_correct_arr
    
    return df

In [ ]:
df.groupby(['testType', 'KnowledgeTag'])['answerCode'].agg(['mean', 'sum', 'std'])

In [ ]:
df['total_test_time'] = (df.test_end_time - df.test_start_time).values / np.timedelta64(1, 's')
df['normalized_total_test_time'] = df.total_test_time / (df.test_itemnum - 1)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,elapsedTime,test_cumcount,test_itemnum,test_num,test_start_time,test_end_time,testType,testNumber,questionNumber,total_test_time,normalized_total_test_time
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,3.0,1,6,0,2020-03-24 00:17:11,2020-03-24 00:17:47,A06,1,1,36.0,7.200000
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,8.0,2,6,0,2020-03-24 00:17:11,2020-03-24 00:17:47,A06,1,2,36.0,7.200000
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,7.0,3,6,0,2020-03-24 00:17:11,2020-03-24 00:17:47,A06,1,3,36.0,7.200000
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,7.0,4,6,0,2020-03-24 00:17:11,2020-03-24 00:17:47,A06,1,4,36.0,7.200000
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,11.0,5,6,0,2020-03-24 00:17:11,2020-03-24 00:17:47,A06,1,5,36.0,7.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,NaN,5,5,0,2020-06-05 06:46:41,2020-06-05 06:50:21,A03,71,5,220.0,55.000000
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,11.0,1,4,0,2020-08-21 01:06:39,2020-08-21 01:08:49,A04,165,1,130.0,43.333333
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,46.0,2,4,0,2020-08-21 01:06:39,2020-08-21 01:08:49,A04,165,2,130.0,43.333333
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,73.0,3,4,0,2020-08-21 01:06:39,2020-08-21 01:08:49,A04,165,3,130.0,43.333333


In [ ]:
testtime_df = df[df['total_test_time'] < 86400*30].groupby('total_test_time').answerCode.mean()
testtime_df

In [ ]:
import collections
temp_df = df.groupby('userID').agg({
    'testType':'unique'
}).reset_index()

temp_df = temp_df['testType'].apply(collections.Counter)
user_type_df =pd.DataFrame.from_records(temp_df).fillna(value=0)

In [ ]:
temp_df = df.groupby(['userID', 'KnowledgeTag']).answerCode.mean().reset_index()
temp_df2 = temp_df.groupby('userID').answerCode.min().reset_index()
temp_df = pd.merge(temp_df, temp_df2, how='left', on='userID')
temp_df = temp_df[temp_df['answerCode_x'] == temp_df['answerCode_y']][['userID','KnowledgeTag']]
temp_df.columns = ['userID', 'hardestTag']

In [143]:
df['shift'] = df.groupby(['userID', 'KnowledgeTag'])['answerCode'].shift(1)
for window_size in [2,3,5]:
    temp_df = df.groupby(['userID', 'KnowledgeTag'])['shift'].rolling(window=window_size, min_periods=1).mean().reset_index()
    df[f'tag_accuracy_trend{window_size}'] = temp_df.sort_values('level_2')['shift']

In [ ]:
temp_df.groupby(['userID', 'KnowledgeTag']).answerCode.rolling(window=3, min_periods=1).mean().reset_index()